In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import (metrics, model_selection, linear_model, preprocessing, ensemble, neighbors)
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
import numpy as np
import pandas as pd
import pprint as pp
import re
#import xgboost as xgb

# import the "crash" data
data = pd.read_csv("my_map_grid.csv",header=7)

# preprocessing
# lowercase: http://stackoverflow.com/a/38931854
data.columns = data.columns.str.lower()
# remove spaces
data.columns = data.columns.str.replace(' ', '_')
# special cases
data.columns = data.columns.str.replace('crash_i_d', 'crash_id')
# remove whateva data

# replace ['No Data','Not Applicable'] with NaN
data.replace(to_replace='No Data', value=np.nan, inplace=True)
data.replace(to_replace='Not Applicable', value=np.nan, inplace=True)
# encode data




In [2]:
data

,crash_id,average_daily_traffic_amount,average_daily_traffic_year,crash_death_count,crash_incapacitating_injury_count,crash_non-incapacitating_injury_count,crash_not_injured_count,crash_possible_injury_count,crash_severity,crash_time,...,longitude,manner_of_collision,medical_advisory_flag,number_of_entering_roads,number_of_lanes,object_struck,road_base_type,speed_limit,street_name,surface_condition
0,14233565,NaN,NaN,0,0,1,0,0,Non-Incapacitating Injury,2023,...,-97.73784246,One Motor Vehicle - Going Straight,No,Other (Explain In Narrative),NaN,NaN,NaN,30,RED RIVER ST,Wet
1,14251326,NaN,NaN,0,0,1,2,0,Non-Incapacitating Injury,1800,...,-97.69914247,One Motor Vehicle - Turning Left,No,Three Entering Roads - T,NaN,NaN,NaN,35,OAK SPRINGS DR,Dry
2,14359985,NaN,NaN,0,0,0,1,1,Possible Injury,635,...,-97.67742598,One Motor Vehicle - Going Straight,No,Other (Explain In Narrative),NaN,NaN,NaN,50,W HOWARD LN,Dry
3,14285346,NaN,NaN,0,0,1,1,0,Non-Incapacitating Injury,1051,...,-97.71535247,One Motor Vehicle - Turning Left,No,Four Entering Roads,NaN,NaN,NaN,40,E 51ST ST,Wet
4,14243394,NaN,NaN,0,1,0,1,0,Incapacitating Injury,538,...,-97.72573519,One Motor Vehicle - Going Straight,No,NaN,NaN,NaN,NaN,40,N LAMAR BLVD,Dry
5,14266345,NaN,NaN,0,0,1,2,0,Non-Incapacitating Injury,1600,...,NaN,One Motor Vehicle - Backing,No,NaN,NaN,NaN,NaN,-1,BURNS ST,Dry
6,14277440,NaN,NaN,0,0,1,0,0,Non-Incapacitating Injury,810,...,-97.73582246,One Motor Vehicle - Going Straight,No,Four Entering Roads,NaN,NaN,NaN,30,NORTHLOOP BLVD W,Dry
7,14251491,NaN,NaN,0,0,0,2,0,Not Injured,1047,...,-97.76571246,One Motor Vehicle - Going Straight,No,Three Entering Roads - T,NaN,NaN,NaN,35,S 1ST ST,Dry
8,14245420,44197,2014,0,1,0,1,0,Incapacitating Injury,1525,...,-97.7516787,One Motor Vehicle - Going Straight,No,Four Entering Roads,4,NaN,Flex Base (Granular),-1,SL0360,Dry
9,14280356,NaN,NaN,0,0,1,1,0,Non-Incapacitating Injury,1419,...,-97.73460246,One Motor Vehicle - Going Straight,No,Four Entering Roads,NaN,NaN,NaN,30,W DEAN KEETON ST,Dry


In [ ]:
# convert 'Wet' 'Dry' to '1' '0'
data['surface_condition'] = data['surface_condition'].factorize()[0]
# DOC: rename col http://stackoverflow.com/a/11346337
data.rename(columns={'surface_condition':'surface_wet'})
# print number of unique
for colname in data.columns:
    print("% 4d : %s" % (len(data[colname].unique()), colname))
# remove data which is has no importance
# better to drop cols with all NaN and convert "unimportant" data to NaN
#  - can't universally decide to drop col just based on uniqueness
# e.g. all of object_struck is 'Not Applicable' and useless, but if surface_condition had only one value "dry" this would be important
# ? for colname in data.columns:
# colname = 'object_struck'
# if(len(data[colname].unique()) == 1):
#   print("-I-: dropping %s for having all homogenous values %s", (colname, data[colname].unique()[0]))
#   data.drop(colname,axis=1,inplace=True)


    

print(data.head())
print(data.info())
if(1):
  data.describe()
  data.hist()
  data.corr().plot() # TODO: seaborn
  plt.show()
else:
  print("-I-: Skipping...")

pairplot_var_list = [
# 'crash_id',
 'average_daily_traffic_amount',
 'average_daily_traffic_year',
 'crash_death_count',
# 'crash_incapacitating_injury_count',
# 'crash_non-incapacitating_injury_count',
# 'crash_not_injured_count',
# 'crash_possible_injury_count',
 'crash_severity',
 'crash_time',
 'crash_year',
 'day_of_week',
# 'intersecting_street_name',
 'intersection_related',
# 'latitude',
 'light_condition',
# 'longitude',
 'manner_of_collision',
 'medical_advisory_flag',
 'number_of_entering_roads',
 'number_of_lanes',
# 'object_struck',
 'road_base_type',
 'speed_limit',
# 'street_name',
 'surface_condition'
 ]

dummies_needed_list = [
 'crash_severity',
 'day_of_week',
 'intersection_related',
 'light_condition',
 'manner_of_collision',
 'number_of_entering_roads',
 'road_base_type',
# 'surface_condition' # factorized
        ]

# tmp disable
if(0):
    sns.pairplot(data, vars=pairplot_var_list)
    plt.show()

# alternative visualisation
datapt = data.pivot_table(values=['crash_death_count','crash_incapacitating_injury_count','crash_non-incapacitating_injury_count'], index=['speed_limit','crash_time'])
print(datapt)

pp.pprint(list(pd.get_dummies(data[dummies_needed_list]).columns))
pp.pprint(list(pd.get_dummies(data[dummies_needed_list]).columns.str.replace('[,\s]+','_').str.lower()))
'''
 'Dark, Lighted', 'dark_lighted_yes'
 'Dark, Not Lighted', 'dark_lighted_no'
 'Dark, Unknown Lighting', 'dark_lighted_unknown'
 'Dawn',
 'Daylight',
 'Dusk',
 'Unknown',
'''